In [1]:
import pyomo.environ as pmo
import numpy as np

np.set_printoptions(precision=15)

class GenericSolver:
    
    def __init__(self, A, b, m):
        # get no of var and constraints
        self.x_card = np.shape(A)[1]
        self.c_card = np.shape(A)[0]
        
        # transform A from matrix to dict
        A_init = {}
        for i in range(self.c_card):
            for j in range(self.x_card):
                A_init[(i+1, j+1)] = A[i, j]
        
        # transform b from vector to dict
        b_init = {}
        for i in range(self.c_card):
            b_init[i+1] = b[i]
            
        # transform m from vector to dict
        m_init = {}
        for i in range(self.x_card):
            m_init[i+1] = m[i]
        
        # define pyomo model
        self.model = pmo.ConcreteModel()
        self.model.n = pmo.RangeSet(1, self.x_card)
        self.model.c = pmo.RangeSet(1, self.c_card)
        self.model.A = pmo.Param(self.model.c, self.model.n, initialize=A_init)
        self.model.b = pmo.Param(self.model.c, initialize=b_init)
        self.model.m = pmo.Param(self.model.n, initialize=m_init)
        self.model.x = pmo.Var(self.model.n, domain=pmo.NonNegativeReals)
        self.model.dual = pmo.Suffix(direction=pmo.Suffix.IMPORT)
        self.model.constraints = pmo.ConstraintList()
        for c in self.model.c:
            self.model.constraints.add(
                sum(self.model.A[c, i] * self.model.x[i] for i in self.model.n) <= self.model.b[c]
            )
        self.model.obj = pmo.Objective(
            expr=sum(self.model.m[i] * self.model.x[i] for i in self.model.n)
        )
        
        # define solver
        self.solverpath = 'C:\\w64\\glpsol'
        self.solver = pmo.SolverFactory('glpk', executable=self.solverpath)
#         self.solver = pmo.SolverFactory('cplex')
    
        # define empty output entities
        self.soln = None
        self.duals = None
    
    def solve(self):
        self.solver.solve(self.model, tee=False)
        self.soln = np.empty([self.x_card])
        for i in range(self.x_card):
            self.soln[i] = self.model.x[i+1].value
        self.duals = np.empty([self.c_card])
        for c in range(self.c_card):
            self.duals[c] = -self.model.dual[self.model.constraints[c+1]]

In [2]:
class RedundancyChecker:
    
    def __init__(self, A, b, tol=1e-7):
        self.A = A
        self.b = b
        self.tol = tol
        
        # get no of var and constraints
        self.x_card = np.shape(A)[1]
        self.c_card = np.shape(A)[0]
        
        # transform A from matrix to dict
        A_init = {}
        for i in range(self.c_card):
            for j in range(self.x_card):
                A_init[(i+1, j+1)] = A[i, j]
        
        # transform b from vector to dict
        b_init = {}
        for i in range(self.c_card):
            b_init[i+1] = b[i]
            
        # define pyomo model
        self.model = pmo.ConcreteModel()
        self.model.n = pmo.RangeSet(1, self.x_card)
        self.model.c = pmo.RangeSet(1, self.c_card)
        self.model.A = pmo.Param(self.model.c, self.model.n, initialize=A_init)
        self.model.b = pmo.Param(self.model.c, initialize=b_init)
        self.model.x = pmo.Var(self.model.n)
        self.model.dual = pmo.Suffix(direction=pmo.Suffix.IMPORT)
        self.model.constraints = pmo.ConstraintList()
        for c in self.model.c:
            self.model.constraints.add(
                sum(self.model.A[c, i] * self.model.x[i] for i in self.model.n) <= self.model.b[c]
            )
        
        # define solver
        self.solverpath = 'C:\\w64\\glpsol'
        self.solver = pmo.SolverFactory('glpk', executable=self.solverpath)
#         self.solver = pmo.SolverFactory('cplex')
    
        # define empty output entities
        self.redundancy = None
        self.reduced_A = None
        self.reduced_b = None
        
    def check(self):
        # for each constraint, delete any old obj, set new obj as Ax of chosen constraint
        # and maximise it.
        # Deactivate the chosen constraint itself.
        # Then check if b-Ax to see if positive (constraint is loose).
        # If so, mark as redundant.
        self.redundancy = np.zeros([self.c_card])
        self.slack = np.zeros([self.c_card])
        for c in self.model.c:
            try:
                self.model.del_component(self.model.obj)
            except:
                pass
            self.model.obj = pmo.Objective(
                expr=-sum(self.model.A[c, i] * self.model.x[i] for i in self.model.n)
            )
            #self.model.constraints[c].deactivate()
            self.solver.solve(self.model, tee=False)
            #self.model.constraints[c].activate()
            
            self.slack[c-1] = self.model.b[c] + pmo.value(self.model.obj)

            if self.model.b[c] + pmo.value(self.model.obj) > self.tol:
                self.redundancy[c-1] = 1

        self.reduced_A = self.A[self.redundancy == 0]
        self.reduced_b = self.b[self.redundancy == 0]

In [3]:
class RegionSolver:
    """
    Returns equations representing x, lambda and boundaries of a math problem in terms of theta.
    self.soln_slope
    self.soln_constant
    self.boundary_slope
    self.boundary_constant
    """
    def __init__(self, A, b, m, theta_count):
        self.A = A
        self.b = b
        self.m = m
        self.theta_count = theta_count
        self.x_count = np.shape(A)[1] - self.theta_count
        self.var_count = np.shape(A)[1]
        self.c_count = np.shape(A)[0]
        
        # returned from _solve_theta
        self.theta = None
        
        # returned from _solve_x
        self.x_problem_A = None
        self.x_problem_b = None
        self.x_problem_theta_cols = None
        self.x = None
        self.duals = None

        # returned from _get_MN
        self.M = None
        self.N = None
        self.MN = None
        
        # returned from _get_soln_params
        self.soln_slope = None
        self.soln_constant = None
        
        # returned from _set_boundaries
        self.boundary_slope = None
        self.boundary_constant = None
        
    def _solve_theta(self):
        # edit b by a little bit to ensure we try somewhere within the region
        modified_b = self.b - 1e-7
        theta_problem = GenericSolver(self.A, modified_b, self.m)
        theta_problem.solve()
        self.theta = theta_problem.soln[-self.theta_count:]
       
    def _solve_x(self):
        # define A without theta, and ignore constraints just for theta
        self.x_problem_A = self.A[:, :(self.var_count - self.theta_count)]

        # define b ignoring constraints just for theta
        self.x_problem_theta_cols = self.A[:, -self.theta_count:]
        self.x_problem_b = self.b - np.dot(self.x_problem_theta_cols, self.theta)
        
        delete_rows = []
        for r in range(self.c_count):
            if np.sum(np.abs(self.x_problem_A[r])) == 0:
                delete_rows.append(r)
        self.x_problem_A = np.delete(self.x_problem_A, delete_rows, axis=0)
        self.x_problem_b = np.delete(self.x_problem_b, delete_rows)
#         # !!!!dirty hack!!!!
#         region_problem.x_problem_b[0]=30000+0.000001
        self.x_problem_theta_cols = np.delete(self.x_problem_theta_cols, delete_rows, axis=0)
        self.x_problem_b_original = np.delete(self.b, delete_rows)
        
        # solve for x, duals
        x_problem = GenericSolver(self.x_problem_A, self.x_problem_b, self.m)
        x_problem.solve()
        self.x = x_problem.soln 
        self.duals = x_problem.duals
    
    def _get_MN(self):
        M_len = self.x_count + np.shape(self.x_problem_A)[0]
        self.M = np.zeros([M_len, M_len])
        self.M[:self.x_count, self.x_count:] = self.x_problem_A.T
        self.M[self.x_count:, :self.x_count] = np.multiply(self.x_problem_A.T, self.duals).T

        # if whole row is zero, multiplier is zero so delete row
        delete_rows = []
        for r in range(M_len):
            if np.sum(np.abs(self.M[r])) == 0:
                delete_rows.append(r)
        self.M = np.delete(self.M, delete_rows, axis=0)    
        self.M = np.delete(self.M, delete_rows, axis=1)
        
        # M has (no of var + no of constraints) rows.
        # For matrices theta_cols and duals, they only have rows equal to no of constraints.
        # Here we want to delete constraints that are redundant, but list delete_rows count in rows of M.
        # So count back no of var to compute rows to delete for theta_cols and duals.
        delete_rows_constraints_only = delete_rows - np.ones(len(delete_rows)) * self.x_count
        delete_rows_constraints_only = delete_rows_constraints_only.astype('int')
        
        # delete redundant rows from theta_cols, duals and N also to ensure non-singular matrix
        reduced_theta_cols = np.delete(self.x_problem_theta_cols, delete_rows_constraints_only, axis=0)
        reduced_duals = np.delete(self.duals, delete_rows_constraints_only)
        
        self.N = np.zeros([np.shape(self.M)[0], self.theta_count])
        self.N[self.x_count:] = np.multiply(reduced_theta_cols.T, reduced_duals).T
        
        MN_result = np.linalg.solve(self.M, self.N)
        self.MN = np.zeros([M_len, self.theta_count])
        kept_rows = np.delete(np.array(range(M_len)), delete_rows)
        
        for i in range(len(kept_rows)):
            self.MN[kept_rows[i], :] = MN_result[i]
        
    def _get_soln_params(self):
        self.soln_slope = -self.MN
        self.soln_constant = np.dot(-self.MN, -self.theta) + np.r_[self.x, self.duals]
        
    def _set_boundaries(self):
        # substitute x = G * theta + H into Ax <= b
        # Means AG * theta + AH <= b
        # A: x_problem_A, remove active constraints
        # b: x_problem_b, remove active constraints
        # G: soln_slope, for x (so remove lambda)
        # H: soln_constant, for x (so remove lambda)
        #
        # Then need to add back the theta theta cols into the constraints. We can use x_problem_theta_cols
        
        # formulate A, b
        sub_A = self.x_problem_A[self.duals == 0.0]        
        sub_b = self.x_problem_b_original[self.duals == 0.0]
        sub_theta_cols = self.x_problem_theta_cols[self.duals == 0.0]
        
        sub_G = self.soln_slope[:self.x_count]
        sub_H = self.soln_constant[:self.x_count]
        
        AG = np.dot(sub_A, sub_G)
        AH = np.dot(sub_A, sub_H)
        
        AG_with_theta_cols = AG + sub_theta_cols
        
        new_rhs = sub_b - AH
        #print('new_rhs = ' + str(new_rhs))
        
        A_theta_only_constraints = np.sum(np.abs(self.A[:, :self.x_count]), axis=1) == 0
        
        boundary_slope = np.concatenate((AG_with_theta_cols, self.A[A_theta_only_constraints][:, -self.theta_count:]), axis=0)
        boundary_constant = np.concatenate((new_rhs, self.b[A_theta_only_constraints]))   
        
        reduction_problem = RedundancyChecker(boundary_slope, boundary_constant)
        reduction_problem.check()

        self.boundary_slope = reduction_problem.reduced_A
        self.boundary_constant = reduction_problem.reduced_b
        
    def solve(self):
        self._solve_theta()
        self._solve_x()
        self._get_MN()
        self._get_soln_params()
        self._set_boundaries()

In [4]:
A = np.array(
    [[0.8, 0.44, -1.0, 0.0],
     [0.05, 0.1, 0.0, -1.0],
     [0.1, 0.36, 0.0, 0.0],
     # x boundaries
     [-1.0, 0.0, 0.0, 0.0],
     [0.0, -1.0, 0.0, 0.0],
     # theta boundaries
     [0.0, 0.0, -1.0, 0.0],
     [0.0, 0.0, 1.0, 0.0],
     [0.0, 0.0, 0.0, -1.0],
     [0.0, 0.0, 0.0, 1.0]]
).astype(np.float64)

b = np.array(
    [24000.0, 2000.0, 6000.0, 0.0, 0.0, 0.0, 6000.0, 0.0, 500.0]
).astype(np.float64)

m = np.array(
    [-8.1, -10.8, 0.0, 0.0]
).astype(np.float64)

theta_count = 2


In [5]:
class ParametricSolver:
    
    def __init__(self, A, b, m, theta_count):
        self.system = {
            'A': A, 
            'b': b,
            'm': m,
            'theta_count': theta_count
        }
        
        self.regions = {}        
    
    def create_region(self, soln_A, soln_b, firm_bound_A, firm_bound_b, flippable_bound_A, flippable_bound_b, solved_status):
        region_def = {
            'soln_A': soln_A,
            'soln_b': soln_b,
            'firm_bound_A': firm_bound_A,
            'firm_bound_b': firm_bound_b,
            'flippable_bound_A': flippable_bound_A,
            'flippable_bound_b': flippable_bound_b,
            'solved_status': solved_status
        }
        self.regions[self.new_index()] = region_def
    
    def solve_original_problem(self):
        original_problem = RegionSolver(
            self.system['A'], 
            self.system['b'], 
            self.system['m'], 
            self.system['theta_count']
        )

        original_problem.solve()
        
        firm_A, firm_b, flippable_A, flippable_b = self.categorise_const(
            original_problem.boundary_slope, 
            original_problem.boundary_constant,
            np.empty([0, self.system['theta_count']]),
            np.empty([0])
        )
        
        self.create_region(
            original_problem.soln_slope,
            original_problem.soln_constant,
            firm_A,
            firm_b,
            flippable_A,
            flippable_b,
            True
        )
        
    def gen_new_regions(self, region_index):
        flippable_A = self.regions[region_index]['flippable_bound_A']
        flippable_b = self.regions[region_index]['flippable_bound_b']
        no_of_new_regions = np.shape(flippable_b)[0]
        
        next_added_A = np.empty(shape=[0, np.shape(flippable_A)[1]])
        next_added_b = np.empty(shape=[0])
        for n in range(no_of_new_regions):
            next_added_A = np.append(next_added_A, [np.multiply(flippable_A[n], -1)], axis=0)
            next_added_b = np.append(next_added_b, [np.multiply(flippable_b[n], -1)])
            self.create_region(
                None,
                None,
                next_added_A,
                next_added_b,
                None,
                None,
                False
            )
#             next_added_A[-1] = np.multiply(next_added_A[-1], -1)
#             next_added_b[-1] = np.multiply(next_added_b[-1], -1)
        
        # now label the considered constraints as firm
        self.regions[region_index]['firm_bound_A'] = np.append(
            self.regions[region_index]['firm_bound_A'],
            self.regions[region_index]['flippable_bound_A'],
            axis=0
        )
        self.regions[region_index]['firm_bound_b'] = np.append(
            self.regions[region_index]['firm_bound_b'],
            self.regions[region_index]['flippable_bound_b']
        )
        self.regions[region_index]['flippable_bound_A'] = None
        self.regions[region_index]['flippable_bound_b'] = None

    
    def solve_region_problem(self, region_index):
        # assumes region is already created in dict regions.
        extended_firm_bound_A = np.concatenate(
            (
                np.zeros(
                    [np.shape(self.regions[region_index]['firm_bound_A'])[0], 
                     np.shape(self.system['A'])[1] - self.system['theta_count']]
                ),
                self.regions[region_index]['firm_bound_A']
            ),
            axis=1
        )
            
        region_problem_A = np.concatenate(
            (
                self.system['A'],
                extended_firm_bound_A
            ),
            axis=0
        )
        
        region_problem_b = np.concatenate(
            (
                self.system['b'],
                self.regions[region_index]['firm_bound_b']
            )
        )
        
        region_problem = RegionSolver(region_problem_A, region_problem_b, self.system['m'], self.system['theta_count'])
        region_problem.solve()
        
        self.regions[region_index]['soln_A'] = region_problem.soln_slope
        self.regions[region_index]['soln_b'] = region_problem.soln_constant
        
        firm_A, firm_b, flippable_A, flippable_b = self.categorise_const(
            region_problem.boundary_slope, 
            region_problem.boundary_constant,
            self.regions[region_index]['firm_bound_A'],
            self.regions[region_index]['firm_bound_b']
        )
    
        self.regions[region_index]['firm_bound_A'] = firm_A
        self.regions[region_index]['firm_bound_b'] = firm_b
        self.regions[region_index]['flippable_bound_A'] = flippable_A
        self.regions[region_index]['flippable_bound_b'] = flippable_b
        self.regions[region_index]['solved_status'] = True
    
    def new_index(self):
        try:
            last_index = list(self.regions.keys())[-1]
            return last_index + 1
        except IndexError:
            return 0
        
    def categorise_const(self, lhs, rhs, firm_lhs, firm_rhs):
        # for bounds generated by solving a region problem, do two comparisons:
        # with the original model, and then
        # with the current firm bounds (probably given by other regions).
        eqn_concat = np.concatenate((lhs, np.array([rhs]).T), axis=1)
        sys_concat = np.concatenate(
            (
                self.system['A'][:, -self.system['theta_count']:], 
                np.array([self.system['b']]).T
            ), 
            axis=1)
        firm_concat = np.concatenate(
            (
                firm_lhs, 
                np.array([firm_rhs]).T,
            ),
            axis=1
        )
        
        firm_A = np.empty(shape=[0, np.shape(lhs)[1]])
        firm_b = np.empty(shape=[0])
        flippable_A = np.empty(shape=[0, np.shape(lhs)[1]])
        flippable_b = np.empty(shape=[0])
        
        for row in range(np.shape(eqn_concat)[0]):
#             print('row=' + str(row))
#             print('eqn=' + str(eqn_concat[row]))
#             print('sys=' + str(sys_concat))
            is_firm = np.logical_or(
                np.any(np.all(eqn_concat[row] == sys_concat, axis=1)),
                np.any(np.all(eqn_concat[row] == firm_concat, axis=1))
            )
            if is_firm:
#                 print('eqn in sys')
                firm_A = np.append(firm_A, np.array([lhs[row]]), axis=0)
                firm_b = np.append(firm_b, np.array([rhs[row]]), axis=0)
            else:
#                 print('eqn flippable')
                flippable_A = np.append(flippable_A, np.array([lhs[row]]), axis=0)
                flippable_b = np.append(flippable_b, np.array([rhs[row]]), axis=0)
        
        return firm_A, firm_b, flippable_A, flippable_b
    
    def reduce_region_bounds(self, region_index):
        firm_concat = np.concatenate(
            (
                self.regions[region_index]['firm_bound_A'], 
                np.array([self.regions[region_index]['firm_bound_b']]).T,
            ),
            axis=1
        )
        
        flippable_concat = np.concatenate(
            (
                self.regions[region_index]['flippable_bound_A'], 
                np.array([self.regions[region_index]['flippable_bound_b']]).T,
            ),
            axis=1
        )

        for row in range(np.shape(firm_concat)[0]):
            if self.regions[region_index]['firm_bound_b'][row] != 0:
                firm_concat[row] = np.divide(firm_concat[row], np.abs(firm_concat[row][-1]))
        
        all_dup_rows = np.empty([0])
        for row in range(np.shape(firm_concat)[0]):
            check_duplicate = np.where(np.all(np.isclose(firm_concat, firm_concat[row]), axis=1))
            
            dup_rows = np.empty([0])
            if np.shape(check_duplicate)[0] > 1:
                dup_rows = check_duplicate[1:]
                
            all_dup_rows = np.append(all_dup_rows, dup_rows)
            
        if np.shape(all_dup_rows)[0] > 0:
            all_dup_rows = np.unique(all_dup_rows)
            self.regions[region_index]['firm_bound_A'] = np.delete(
                self.regions[region_index]['firm_bound_A'], 
                all_dup_rows,
                axis=0
            )
            self.regions[region_index]['firm_bound_b'] = np.delete(
                self.regions[region_index]['firm_bound_b'], 
                all_dup_rows
            )
        
        
        # flippable now
        for row in range(np.shape(flippable_concat)[0]):
            if self.regions[region_index]['flippable_bound_b'][row] != 0:
                flippable_concat[row] = np.divide(flippable_concat[row], np.abs(flippable_concat[row][-1]))
        
        all_dup_rows = np.empty([0])
        for row in range(np.shape(flippable_concat)[0]):
            check_duplicate = np.where(np.all(np.isclose(flippable_concat, flippable_concat[row]), axis=1))
            
            dup_rows = np.empty([0])
            if np.shape(check_duplicate)[0] > 1:
                dup_rows = check_duplicate[1:]

            all_dup_rows = np.append(all_dup_rows, dup_rows)
            
        if np.shape(all_dup_rows)[0] > 0:
            all_dup_rows = np.unique(all_dup_rows)
            self.regions[region_index]['flippable_bound_A'] = np.delete(
                self.regions[region_index]['flippable_bound_A'], 
                all_dup_rows,
                axis=0
            )
            self.regions[region_index]['flippable_bound_b'] = np.delete(
                self.regions[region_index]['flippable_bound_b'], 
                all_dup_rows
            )
        
        # check if any flippable constraint is in firm list
        all_dup_rows = np.empty([0])
        for row in range(np.shape(flippable_concat)[0]):
            check_duplicate = np.where(np.all(np.isclose(firm_concat, flippable_concat[row]), axis=1))
            
            dup_rows = check_duplicate
            
            if np.shape(dup_rows)[0] > 0:
                all_dup_rows = np.append(all_dup_rows, row)
            
        if np.shape(all_dup_rows)[0] > 0:
            all_dup_rows = np.unique(all_dup_rows)
            self.regions[region_index]['flippable_bound_A'] = np.delete(
                self.regions[region_index]['flippable_bound_A'], 
                all_dup_rows,
                axis=0
            )
            self.regions[region_index]['flippable_bound_b'] = np.delete(
                self.regions[region_index]['flippable_bound_b'], 
                all_dup_rows
            )
        

# !!!!! not done yet - in the categorisation also need to compare with current firm constraints !!!! need to consider scaling
# !!!!! also need to make sure constraint added to flippable is compared to others in flippable.
# !!!!! somehow the flippable constraint gets moved from firm to flippable for region 1.
# two functionalities 1. to eliminiate equivalent rows in an A, b set, 2. to find out what's in current firm and don't add this to flippable
# 2 is done!
# 1 is done! althoguh a bit ugly

        
        

Note two things:
- In method reduce_region_bounds, we use np.isclose to compare row to a matrix.

In [6]:
parametric_problem = ParametricSolver(A, b, m, theta_count)
parametric_problem.solve_original_problem()

In [7]:
parametric_problem.gen_new_regions(0)

In [8]:
parametric_problem.solve_region_problem(1)
parametric_problem.regions

{0: {'soln_A': array([[ 1.475409836065574, -0.               ],
         [-0.409836065573771, -0.               ],
         [-0.               , -0.               ],
         [-0.               , -0.               ],
         [-0.               , -0.               ],
         [-0.               , -0.               ],
         [-0.               , -0.               ]]),
  'soln_b': array([2.459016393442620e+04, 9.836065573770491e+03,
         7.524590163934430e+00, 0.000000000000000e+00,
         2.080327868852460e+01, 0.000000000000000e+00,
         0.000000000000000e+00]),
  'firm_bound_A': array([[-1.               ,  0.               ],
         [ 1.               ,  0.               ],
         [ 0.               ,  1.               ],
         [ 0.032786885245902, -1.               ]]),
  'firm_bound_b': array([   0.             , 6000.             ,  500.             ,
         -213.1147540983593]),
  'flippable_bound_A': None,
  'flippable_bound_b': None,
  'solved_status': True

In [9]:
parametric_problem.reduce_region_bounds(1)


C:\Users\user1\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:271: DeprecationWarning: using a non-integer array as obj in delete will result in an error in the future
C:\Users\user1\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:275: DeprecationWarning: using a non-integer array as obj in delete will result in an error in the future


In [10]:
parametric_problem.regions

{0: {'soln_A': array([[ 1.475409836065574, -0.               ],
         [-0.409836065573771, -0.               ],
         [-0.               , -0.               ],
         [-0.               , -0.               ],
         [-0.               , -0.               ],
         [-0.               , -0.               ],
         [-0.               , -0.               ]]),
  'soln_b': array([2.459016393442620e+04, 9.836065573770491e+03,
         7.524590163934430e+00, 0.000000000000000e+00,
         2.080327868852460e+01, 0.000000000000000e+00,
         0.000000000000000e+00]),
  'firm_bound_A': array([[-1.               ,  0.               ],
         [ 1.               ,  0.               ],
         [ 0.               ,  1.               ],
         [ 0.032786885245902, -1.               ]]),
  'firm_bound_b': array([   0.             , 6000.             ,  500.             ,
         -213.1147540983593]),
  'flippable_bound_A': None,
  'flippable_bound_b': None,
  'solved_status': True